In [43]:
pip install prophet

Note: you may need to restart the kernel to use updated packages.


In [118]:
#library required
import psycopg2

from sqlalchemy import create_engine
from sqlalchemy import text
import os
import time 
from datetime import datetime
import io 
import sys

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error


In [117]:
#Load the data

# Define the file paths
file_path_1 = r"C:\Users\Poovesh Kumar M\Downloads\Planning\1_Data\3_Sep_05_09_2024\majestic pure_Amazon_com_weekly_data.csv"
file_path_2 = r"C:\Users\Poovesh Kumar M\Downloads\Planning\1_Data\3_Sep_05_09_2024\ASIN_classification_weekly.csv"
# file_path_3 = r"C:\Users\Poovesh Kumar M\Downloads\Planning\1_Data\3_Sep_05_09_2024\file3.csv"
# file_path_4 = r"C:\Users\Poovesh Kumar M\Downloads\Planning\1_Data\3_Sep_05_09_2024\file4.csv"

# Read the CSV files into DataFrames
df_base1 = pd.read_csv(file_path_1)
df_asin_class = pd.read_csv(file_path_2)


In [57]:
# Display the DataFrame
print(df_weekly)

      velocity_bucket        asin Week_Start_date Week_End_date  \
0                1_SF  B00PMR3QF2      2022-09-26    2022-10-02   
1                1_SF  B00PMR3QF2      2022-10-03    2022-10-09   
2                1_SF  B00PMR3QF2      2022-10-10    2022-10-16   
3                1_SF  B00PMR3QF2      2022-10-17    2022-10-23   
4                1_SF  B00PMR3QF2      2022-10-24    2022-10-30   
...               ...         ...             ...           ...   
23741            4_SM  B0D83GG4WB      2024-08-05    2024-08-11   
23742            4_SM  B0D83GG4WB      2024-08-12    2024-08-18   
23743            4_SM  B0D83GG4WB      2024-08-19    2024-08-25   
23744            4_SM  B0D83GG4WB      2024-08-26    2024-09-01   
23745            4_SM  B0D83GG4WB      2024-09-02    2024-09-08   

       organic_gmv_units  inorganic_gmv_units  gmv_units   asp_week  \
0               220.8734             145.9917        368  18.886359   
1               756.0293             464.0168       1

In [86]:
# Filtering input data:

df_asin_class1 = df_asin_class[['asin', 'classification']]

df_base = pd.merge(df_base1, df_asin_class1, on ='asin')

df_weekly = df_base

df_weekly.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/df_weekly_base.csv', index=False)

# Convert 'Week_Start_date' to datetime format
df_weekly['Week_Start_date'] = pd.to_datetime(df_weekly['Week_Start_date'])

# Replace NaN values in 'asp_week' column with 0
df_weekly['asp_week'] = df_weekly['asp_week'].fillna(0)

# Filter df_weekly to only include rows where 'classification' is 'Smooth' or 'Intermittent'
df_weekly = df_weekly[df_weekly['classification'].isin(['Smooth', 'Intermittent'])]


In [61]:
df_weekly

,velocity_bucket,asin,Week_Start_date,Week_End_date,organic_gmv_units,inorganic_gmv_units,gmv_units,asp_week,ad_spend,inventory_Availability%,gmv,inorganic_gmv,organic_gmv,inventory_available,classification
0,1_SF,B00PMR3QF2,2022-09-26,2022-10-02,220.8734,145.9917,368,18.886359,1626.180000,0.000000,6950.18,2757.6640,4171.0360,0,Smooth
1,1_SF,B00PMR3QF2,2022-10-03,2022-10-09,756.0293,464.0168,1227,18.895069,4042.875300,0.000000,23184.25,8768.5915,14284.1985,0,Smooth
2,1_SF,B00PMR3QF2,2022-10-10,2022-10-16,834.9785,813.4476,1655,16.091245,6729.707800,0.000000,26631.01,13023.9672,13496.8928,0,Smooth
3,1_SF,B00PMR3QF2,2022-10-17,2022-10-23,883.1982,597.8767,1489,16.000302,4171.113800,0.000000,23824.45,9565.7531,14131.8569,0,Smooth
4,1_SF,B00PMR3QF2,2022-10-24,2022-10-30,1034.3934,694.0630,1742,16.050614,4477.239100,0.000000,27960.17,11141.9337,16600.6563,0,Smooth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23731,4_SM,B0D811N8HV,2024-08-05,2024-08-11,102.5283,0.9204,106,9.220189,6.148061,100.000000,977.34,8.9900,943.5900,12717,Smooth
23732,4_SM,B0D811N8HV,2024-08-12,2024-08-18,103.6639,0.0000,104,9.013077,0.000000,100.000000,937.36,0.0000,934.2400,12485,Smooth
23733,4_SM,B0D811N8HV,2024-08-19,2024-08-25,92.7710,0.0000,94,9.111277,0.000000,100.000000,856.46,0.0000,844.7000,11720,Smooth
23734,4_SM,B0D811N8HV,2024-08-26,2024-09-01,102.1923,0.9864,105,9.150857,1.462254,100.000000,960.84,8.9900,934.9600,10909,Smooth


In [69]:
#creating New DF 
df_bs_temp = df_weekly
# Display the first few rows of the filtered DataFrame
print(df_bs_temp.head())

#creating ASINx VB for mapping purpose
# Remove rows where 'asin' is null
df_asin_vb = df_bs_temp.dropna(subset=['asin'])

# Select distinct 'asin' x 'velocity_bucket' x classification:
df_asin_vb = df_asin_vb[['asin', 'velocity_bucket', 'classification']].drop_duplicates()

# Display the resulting DataFrame
print("Distinct asin x velocity_bucket:")
print(df_asin_vb.head())




  velocity_bucket        asin Week_Start_date Week_End_date  \
0            1_SF  B00PMR3QF2      2022-09-26    2022-10-02   
1            1_SF  B00PMR3QF2      2022-10-03    2022-10-09   
2            1_SF  B00PMR3QF2      2022-10-10    2022-10-16   
3            1_SF  B00PMR3QF2      2022-10-17    2022-10-23   
4            1_SF  B00PMR3QF2      2022-10-24    2022-10-30   

   organic_gmv_units  inorganic_gmv_units  gmv_units   asp_week   ad_spend  \
0           220.8734             145.9917        368  18.886359  1626.1800   
1           756.0293             464.0168       1227  18.895069  4042.8753   
2           834.9785             813.4476       1655  16.091245  6729.7078   
3           883.1982             597.8767       1489  16.000302  4171.1138   
4          1034.3934             694.0630       1742  16.050614  4477.2391   

   inventory_Availability%       gmv  inorganic_gmv  organic_gmv  \
0                      0.0   6950.18      2757.6640    4171.0360   
1               

# Define the Outlier Treatment Function 
# Plotting Before and After Outlier Treatment

# Store original DataFrame before treatment
df_weekly_original = df_weekly.copy()

# Define the Outlier Treatment Function
def replace_outliers_with_previous_week(group, column):
    Q1 = group[column].quantile(0.25)
    Q3 = group[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outliers = (group[column] < lower_bound) | (group[column] > upper_bound)
    
    # Replace outliers with the previous week's value
    group[column] = np.where(outliers, group[column].shift(1), group[column])
    
    # In case the first value is an outlier and has no previous week to reference, we can forward fill
    group[column] = group[column].fillna(method='ffill')
    
    return group

# Apply the Outlier Treatment
df_weekly = df_weekly.groupby('asin').apply(replace_outliers_with_previous_week, 'gmv_units').reset_index(drop=True)
df_weekly = df_weekly.groupby('asin').apply(replace_outliers_with_previous_week, 'organic_gmv_units').reset_index(drop=True)
df_weekly = df_weekly.groupby('asin').apply(replace_outliers_with_previous_week, 'inorganic_gmv_units').reset_index(drop=True)

# Plotting Before and After Outlier Treatment
def plot_outlier_treatment(df_before, df_after, column, asin_list):
    for asin in asin_list:
        df_before_asin = df_before[df_before['asin'] == asin]
        df_after_asin = df_after[df_after['asin'] == asin]
        
        plt.figure(figsize=(14, 6))
        plt.plot(df_before_asin['Week_Start_date'], df_before_asin[column], marker='o', color='red', label=f'{column} Before Treatment')
        plt.plot(df_after_asin['Week_Start_date'], df_after_asin[column], marker='o', color='blue', label=f'{column} After Treatment')
        plt.title(f"{column} Before and After Outlier Treatment for ASIN: {asin}")
        plt.xlabel("Week Start Date")
        plt.ylabel(column)
        plt.legend()
        plt.grid(True)
        plt.show()

# Use the actual list of ASINs you want to plot
asin_to_plot = ['B07JD9FS28', 'B079ZN8SC9', 'B09S26ZF1N', 'B09PZSKJ5N']
plot_outlier_treatment(df_weekly_original, df_weekly, 'gmv_units', asin_to_plot)
plot_outlier_treatment(df_weekly_original, df_weekly, 'organic_gmv_units', asin_to_plot)
plot_outlier_treatment(df_weekly_original, df_weekly, 'inorganic_gmv_units', asin_to_plot)


In [70]:
# Split the data into training and testing datasets
train_df = df_weekly[df_weekly['Week_Start_date'] <= '2024-04-29']
test_df = df_weekly[(df_weekly['Week_Start_date'] >= '2024-05-06') & (df_weekly['Week_Start_date'] <= '2024-08-26')]


In [67]:
print("train_df:", train_df)


train_df:       velocity_bucket        asin Week_Start_date Week_End_date  \
0                1_SF  B00PMR3QF2      2022-09-26    2022-10-02   
1                1_SF  B00PMR3QF2      2022-10-03    2022-10-09   
2                1_SF  B00PMR3QF2      2022-10-10    2022-10-16   
3                1_SF  B00PMR3QF2      2022-10-17    2022-10-23   
4                1_SF  B00PMR3QF2      2022-10-24    2022-10-30   
...               ...         ...             ...           ...   
23473            4_SM  B0CXDRY3WC      2024-04-08    2024-04-14   
23474            4_SM  B0CXDRY3WC      2024-04-15    2024-04-21   
23475            4_SM  B0CXDRY3WC      2024-04-22    2024-04-28   
23476            4_SM  B0CXDRY3WC      2024-04-29    2024-05-05   
23553            4_SM  B0D1XNW89X      2024-04-29    2024-05-05   

       organic_gmv_units  inorganic_gmv_units  gmv_units   asp_week  \
0               220.8734             145.9917        368  18.886359   
1               756.0293             464.01

In [66]:
print("test_df",test_df)

test_df       velocity_bucket        asin Week_Start_date Week_End_date  \
84               1_SF  B00PMR3QF2      2024-05-06    2024-05-12   
85               1_SF  B00PMR3QF2      2024-05-13    2024-05-19   
86               1_SF  B00PMR3QF2      2024-05-20    2024-05-26   
87               1_SF  B00PMR3QF2      2024-05-27    2024-06-02   
88               1_SF  B00PMR3QF2      2024-06-03    2024-06-09   
...               ...         ...             ...           ...   
23730            4_SM  B0D811N8HV      2024-07-29    2024-08-04   
23731            4_SM  B0D811N8HV      2024-08-05    2024-08-11   
23732            4_SM  B0D811N8HV      2024-08-12    2024-08-18   
23733            4_SM  B0D811N8HV      2024-08-19    2024-08-25   
23734            4_SM  B0D811N8HV      2024-08-26    2024-09-01   

       organic_gmv_units  inorganic_gmv_units  gmv_units   asp_week  \
84             2279.6918            2944.6377       5272  10.126884   
85              988.9535             775.0357

#version 3: day level x holidays
# GMV_units forecasting

import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, r2_score
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

# Define exogenous variables
exogenous_vars = ['asp_week', 'ad_spend']

# Prepare dictionaries to store results
X_train_fb2 = {}
X_test_fb2 = {}
y_test_fb_pred2 = {}

# Generate U.S. holidays DataFrame
cal = calendar()
holidays = cal.holidays(start=train_df['Week_Start_date'].min(), end=test_df['Week_Start_date'].max(), return_name=True)
holiday_df = pd.DataFrame(data=holidays, columns=['holiday'])
holiday_df = holiday_df.reset_index().rename(columns={'index': 'ds'})

for asin in train_df['asin'].unique():
    print(f"Processing ASIN: {asin}")
    
    # Filter the training data for the current ASIN
    asin_train_data = train_df[train_df['asin'] == asin]
    asin_test_data = test_df[test_df['asin'] == asin]
    
    if asin_train_data.empty or asin_test_data.empty:
        print(f"No data found for ASIN: {asin}")
        continue
    
    # Prepare the training data for Prophet
    X_train_fb2[asin] = pd.DataFrame()
    X_train_fb2[asin]['ds'] = asin_train_data['Week_Start_date']
    X_train_fb2[asin]['y'] = asin_train_data['gmv_units']
    
    # Add exogenous variables
    X_train_fb2[asin][exogenous_vars] = asin_train_data[exogenous_vars]
    
    # Ensure data types are correct
    X_train_fb2[asin]['ds'] = pd.to_datetime(X_train_fb2[asin]['ds'])
    X_train_fb2[asin]['y'] = X_train_fb2[asin]['y'].astype(float)
    
    # Initialize the Prophet model with holidays
    model = Prophet(changepoint_prior_scale=0.03, holidays=holiday_df)
    
    # Add exogenous variables as regressors with standardization
    for var in exogenous_vars:
        model.add_regressor(var, standardize=True)
    
    # Fit the model
    try:
        model.fit(X_train_fb2[asin])
    except ValueError as e:
        print(f"Error fitting model for ASIN {asin}: {e}")
        continue
    
    # Prepare the test data for Prophet
    X_test_fb2[asin] = pd.DataFrame()
    X_test_fb2[asin]['ds'] = asin_test_data['Week_Start_date']
    X_test_fb2[asin][exogenous_vars] = asin_test_data[exogenous_vars]
    
    # Ensure test data type is correct
    X_test_fb2[asin]['ds'] = pd.to_datetime(X_test_fb2[asin]['ds'])
    
    # Make predictions
    try:
        forecast_train = model.predict(X_train_fb2[asin].drop(columns=['y']))
        forecast_test = model.predict(X_test_fb2[asin])
    except ValueError as e:
        print(f"Error generating forecast for ASIN {asin}: {e}")
        continue
    
    # Store the predictions
    y_test_fb_pred2[asin] = forecast_test['yhat'].values
    y_test_fb_pred2[asin][y_test_fb_pred2[asin]<0] = 0
    y_test_fb_pred2[asin] = y_test_fb_pred2[asin].round(0).astype('int64')
    
# Create DataFrame with actual and forecasted values
df_results = []
for asin in y_test_fb_pred2:
    asin_test_data = test_df[test_df['asin'] == asin]
    df_results.append(pd.DataFrame({
        'asin': asin,
        'Week_Start_date': asin_test_data['Week_Start_date'],
        'actual_outlier_treated': asin_test_data['gmv_units'],
        'forecasted_value': y_test_fb_pred2[asin]
    }))

df_results = pd.concat(df_results, ignore_index=True)


In [68]:
# Ensure 'asin' column exists
print("Columns in train_df:", train_df.columns)
print("Columns in test_df:", test_df.columns)


Columns in train_df: Index(['velocity_bucket', 'asin', 'Week_Start_date', 'Week_End_date',
       'organic_gmv_units', 'inorganic_gmv_units', 'gmv_units', 'asp_week',
       'ad_spend', 'inventory_Availability%', 'gmv', 'inorganic_gmv',
       'organic_gmv', 'inventory_available', 'classification'],
      dtype='object')
Columns in test_df: Index(['velocity_bucket', 'asin', 'Week_Start_date', 'Week_End_date',
       'organic_gmv_units', 'inorganic_gmv_units', 'gmv_units', 'asp_week',
       'ad_spend', 'inventory_Availability%', 'gmv', 'inorganic_gmv',
       'organic_gmv', 'inventory_available', 'classification'],
      dtype='object')


In [71]:
#version 2:  
# GMV_units forecasting
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, r2_score

# Define exogenous variables
exogenous_vars = ['asp_week', 'ad_spend']

# Prepare dictionaries to store results
X_train_fb2 = {}
X_test_fb2 = {}
y_test_fb_pred2 = {}

for asin in train_df['asin'].unique():
    print(f"Processing ASIN: {asin}")
    
    # Filter the training data for the current ASIN
    asin_train_data = train_df[train_df['asin'] == asin]
    asin_test_data = test_df[test_df['asin'] == asin]
    
    if asin_train_data.empty or asin_test_data.empty:
        print(f"No data found for ASIN: {asin}")
        continue
    
    # Prepare the training data for Prophet
    X_train_fb2[asin] = pd.DataFrame()
    X_train_fb2[asin]['ds'] = asin_train_data['Week_Start_date']
    X_train_fb2[asin]['y'] = asin_train_data['gmv_units']
    
    # Add exogenous variables
    X_train_fb2[asin][exogenous_vars] = asin_train_data[exogenous_vars]
    
    # Ensure data types are correct
    X_train_fb2[asin]['ds'] = pd.to_datetime(X_train_fb2[asin]['ds'])
    X_train_fb2[asin]['y'] = X_train_fb2[asin]['y'].astype(float)
    
    # Initialize the Prophet model
    model = Prophet(changepoint_prior_scale=0.03)
    
    # Prophet(interval_width=0.95, yearly_seasonality='auto', weekly_seasonality='auto',changepoint_prior_scale=0.03)
    
    # Add exogenous variables as regressors with standardization
    for var in exogenous_vars:
        model.add_regressor(var, standardize=True)
    
    # Fit the model
    try:
        model.fit(X_train_fb2[asin])
    except ValueError as e:
        print(f"Error fitting model for ASIN {asin}: {e}")
        continue
    
    # Prepare the test data for Prophet
    X_test_fb2[asin] = pd.DataFrame()
    X_test_fb2[asin]['ds'] = asin_test_data['Week_Start_date']
    X_test_fb2[asin][exogenous_vars] = asin_test_data[exogenous_vars]
    
    # Ensure test data type is correct
    X_test_fb2[asin]['ds'] = pd.to_datetime(X_test_fb2[asin]['ds'])
    
    # Make predictions
    try:
        forecast_train = model.predict(X_train_fb2[asin].drop(columns=['y']))
        forecast_test = model.predict(X_test_fb2[asin])
    except ValueError as e:
        print(f"Error generating forecast for ASIN {asin}: {e}")
        continue
    
    # Store the predictions
    y_test_fb_pred2[asin] = forecast_test['yhat'].values
    y_test_fb_pred2[asin][y_test_fb_pred2[asin] < 0] = 0
    y_test_fb_pred2[asin] = y_test_fb_pred2[asin].round(0).astype('int64')

# Create DataFrame with actual and forecasted values
df_results = []
for asin in y_test_fb_pred2:
    asin_test_data = test_df[test_df['asin'] == asin]
    df_results.append(pd.DataFrame({
        'asin': asin,
        'Week_Start_date': asin_test_data['Week_Start_date'],
        'actual_outlier_treated': asin_test_data['gmv_units'],
        'forecasted_value': y_test_fb_pred2[asin]
    }))

df_results = pd.concat(df_results, ignore_index=True)



Processing ASIN: B00PMR3QF2


16:46:33 - cmdstanpy - INFO - Chain [1] start processing
16:46:33 - cmdstanpy - INFO - Chain [1] done processing
16:46:34 - cmdstanpy - INFO - Chain [1] start processing


Processing ASIN: B00PV15BPW


16:46:34 - cmdstanpy - INFO - Chain [1] done processing
16:46:34 - cmdstanpy - INFO - Chain [1] start processing


Processing ASIN: B00QR6SS6O


...

Processing ASIN: B0CKVX37KH


16:48:30 - cmdstanpy - INFO - Chain [1] done processing
16:48:30 - cmdstanpy - INFO - Chain [1] start processing


Processing ASIN: B0CNVLTSDC


16:48:30 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B0CXDH2LPP
Processing ASIN: B0CXDRY3WC
Processing ASIN: B0D1XNW89X
Error fitting model for ASIN B0D1XNW89X: Dataframe has less than 2 non-NaN rows.


In [122]:
X_test_fb2

{'B00PMR3QF2':             ds   asp_week      ad_spend
 84  2024-05-06  10.126884  12560.099224
 85  2024-05-13  15.971567   4425.383745
 86  2024-05-20  16.061609   1647.974694
 87  2024-05-27  15.930057   3987.756753
 88  2024-06-03  10.080928   9075.009429
 89  2024-06-10  16.619758   3670.944528
 90  2024-06-17  16.084507   3980.153666
 91  2024-06-24  16.074104   4671.446792
 92  2024-07-01  16.098197   4678.601926
 93  2024-07-08  16.103940   4554.415462
 94  2024-07-15  11.067184   7613.111631
 95  2024-07-22  16.101222   3691.878969
 96  2024-07-29  16.073041   3090.873147
 97  2024-08-05  15.992948   2858.102195
 98  2024-08-12  10.264154   6993.106454
 99  2024-08-19  10.251240   5708.101631
 100 2024-08-26  10.200160   3010.208348,
 'B00PV15BPW':             ds   asp_week     ad_spend
 186 2024-05-06  15.005443  4581.378816
 187 2024-05-13  15.036134  6346.571878
 188 2024-05-20  15.011352  4883.181064
 189 2024-05-27  15.030352  5712.238863
 190 2024-06-03  14.946158  6187.

In [85]:
# Storing Results in a DataFrame and Calculating Metrics:

df_weekly1 = df_weekly[['asin', 'Week_Start_date','gmv_units']]

df_results1 =  pd.merge(df_results, df_weekly1, left_on=['asin', 'Week_Start_date'],
                     right_on=['asin', 'Week_Start_date'])

df_results2 = df_asin_vb.merge(df_results1,left_on='asin', right_on='asin', how='right')

# Calculate Absolute Percentage Error
df_results2['absolute_percentage_error'] = abs((df_results2['forecasted_value'] - df_results2['gmv_units']) / df_results2['gmv_units'])

# Calculate Absolute Errors
df_results2['absolute_error'] = abs(df_results2['forecasted_value'] - df_results2['gmv_units'])

# Calculate overall WAPE
overall_wape = df_results2['absolute_error'].sum() / df_results2['gmv_units'].sum()
print(f"\nOverall WAPE: {overall_wape:.2%}")

# df_results2

# Calculate WAPE for each date and velocity bucket
wape_per_date = df_results2.groupby(['Week_Start_date']).apply(
    lambda x: x['absolute_error'].sum() / x['gmv_units'].sum()
).reset_index(name='WAPE')

# Display WAPE
print("\nWAPE for each forecast date:")
print(wape_per_date)

# Save the forecast results to a CSV file
wape_per_date.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_GMV_units.csv', index=False)


# Calculate WAPE for each date and velocity bucket
wape_per_date_bucket = df_results2.groupby(['Week_Start_date', 'velocity_bucket']).apply(
    lambda x: x['absolute_error'].sum() / x['gmv_units'].sum()
).reset_index(name='WAPE')

# Display WAPE
print("\nWAPE for each forecast date x velocity buckets:")
print(wape_per_date_bucket)

# Save the forecast results to a CSV file
wape_per_date_bucket.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_Velocity_buckets_GMV_units.csv', index=False)


# Calculate WAPE : week x class
wape_per_date_asin_class = df_results2.groupby(['Week_Start_date', 'classification']).apply(
    lambda x: x['absolute_error'].sum() / x['gmv_units'].sum()
).reset_index(name='WAPE')

# Display WAPE
print("\nWAPE for each asin x class:")
print(wape_per_date_bucket)

# Save the forecast results to a CSV file
wape_per_date_asin_class.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_ASIN_CLASS_WAPE_buckets_GMV_units.csv', index=False)


# Calculate WAPE for asin :
wape_per_asin = df_results2.groupby(['asin']).apply(
    lambda x: x['absolute_error'].sum() / x['gmv_units'].sum()
).reset_index(name='WAPE')

# Display WAPE
print("\nWAPE for each asin:")
print(wape_per_date_bucket)

# Save the forecast results to a CSV file
wape_per_asin.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_ASIN_WAPE_GMV_units.csv', index=False)



Overall WAPE: 29.66%

WAPE for each forecast date:
   Week_Start_date      WAPE
0       2024-05-06  0.152855
1       2024-05-13  0.197388
2       2024-05-20  0.419111
3       2024-05-27  0.254127
4       2024-06-03  0.190623
5       2024-06-10  0.252382
6       2024-06-17  0.367360
7       2024-06-24  0.368320
8       2024-07-01  0.354246
9       2024-07-08  0.303336
10      2024-07-15  0.333821
11      2024-07-22  0.284076
12      2024-07-29  0.360756
13      2024-08-05  0.287463
14      2024-08-12  0.254564
15      2024-08-19  0.293513
16      2024-08-26  0.328266

WAPE for each forecast date x velocity buckets:
   Week_Start_date velocity_bucket      WAPE
0       2024-05-06            1_SF  0.093004
1       2024-05-06             2_F  0.112408
2       2024-05-06             3_M  0.177761
3       2024-05-06            4_SM  0.300433
4       2024-05-13            1_SF  0.186249
..             ...             ...       ...
63      2024-08-19            4_SM  0.566397
64      2024-08-2

C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1020052734.py:50: RuntimeWarning: divide by zero encountered in scalar divide
  lambda x: x['absolute_error'].sum() / x['gmv_units'].sum()
C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1020052734.py:63: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: x['absolute_error'].sum() / x['gmv_units'].sum()
C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1020052734.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  lambda x: x['absolute_error'].sum() / x['gmv_units'].sum()


In [83]:
df_weekly1

,asin,Week_Start_date,gmv_units
0,B00PMR3QF2,2022-09-26,368
1,B00PMR3QF2,2022-10-03,1227
2,B00PMR3QF2,2022-10-10,1655
3,B00PMR3QF2,2022-10-17,1489
4,B00PMR3QF2,2022-10-24,1742
...,...,...,...
23731,B0D811N8HV,2024-08-05,106
23732,B0D811N8HV,2024-08-12,104
23733,B0D811N8HV,2024-08-19,94
23734,B0D811N8HV,2024-08-26,105


In [84]:
# Result Dataframe: GMV units

sf_base = df_weekly[['velocity_bucket','classification','asin','Week_Start_date', 'gmv_units']]

sf_base = sf_base[(sf_base['Week_Start_date'] >= '2024-05-06') & (sf_base['Week_Start_date'] <= '2024-08-26')]

sf_base = sf_base.rename(columns={'gmv_units': 'sales'})

df_results3 = df_results2.rename(columns={
    'absolute_percentage_error': 'abs_per_error_sales',
    'absolute_error': 'absolute_error_sales',
    'actual_outlier_treated': 'outlier_treated_sales',
    'Week_Start_date': 'forecast_date',
    'forecasted_value':'forecasted_sales'
    
})

df_results3 = df_results3[['asin', 'forecast_date','forecasted_sales','abs_per_error_sales', 'absolute_error_sales']]

#merging all df

sf_data = pd.merge(sf_base, df_results3, left_on=['asin', 'Week_Start_date'], right_on=['asin', 'forecast_date'])


# Display the first few rows of the results
sf_data = sf_data [['velocity_bucket','classification','asin','Week_Start_date','sales','forecasted_sales','abs_per_error_sales',
                    'absolute_error_sales',]]

# Save the forecast results to a CSV file
sf_data.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Final_Weekly_forecast_sales.csv', index=False)


In [119]:
# WAPE: planned vs actual vs forecast

file_path_3 = r"C:\Users\Poovesh Kumar M\Downloads\Planning\1_Data\3_Sep_05_09_2024\WAPE_Buckets_Forecast_data_aug.csv"
df_data_wape = pd.read_csv(file_path_3)



# Calculate Absolute Errors
df_data_wape['absolute_error_AP'] = abs(df_data_wape['Actual'] - df_data_wape['planned_qty'])
df_data_wape['absolute_error_AF'] = abs(df_data_wape['Actual'] - df_data_wape['forecasted_sales'])
df_data_wape['absolute_error_PF'] = abs(df_data_wape['planned_qty'] - df_data_wape['forecasted_sales'])

# # Calculate overall WAPE
# overall_wape = df_data_wape2['absolute_error'].sum() / df_data_wape2['gmv_units'].sum()
# print(f"\nOverall WAPE: {overall_wape:.2%}")

# Calculate WAPE for asin :
wape_per_asin_AP = df_data_wape.groupby(['asin']).apply(
    lambda x: x['absolute_error_AP'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')

# Calculate WAPE for asin :
wape_per_asin_AF = df_data_wape.groupby(['asin']).apply(
    lambda x: x['absolute_error_AF'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')

# Calculate WAPE for asin :
wape_per_asin_PF = df_data_wape.groupby(['asin']).apply(
    lambda x: x['absolute_error_PF'].sum() / x['Actual'].sum()
).reset_index(name='WAPE')

# Display WAPE
print("\nWAPE for each asin:")
print(wape_per_asin_PF)

# Save the forecast results to a CSV file
# wape_per_asin.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_ASIN_WAPE_GMV_units.csv', index=False)



WAPE for each asin:
           asin      WAPE
0    B00PMR3QF2  0.093773
1    B00PV15BPW  0.582272
2    B00Q96XGUU  0.000000
3    B00QR6SS6O  0.074151
4    B00QR7FTLU  0.000000
..          ...       ...
276  B0CHJCNSDV  0.000000
277  B0CHJGH6BJ  0.000000
278  B0CKBWC3DG  0.000000
279  B0CKXTLXSX  0.000000
280  B0CNVL174K  0.000000

[281 rows x 2 columns]


C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1388450053.py:19: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: x['absolute_error_AP'].sum() / x['Actual'].sum()
C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1388450053.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  lambda x: x['absolute_error_AP'].sum() / x['Actual'].sum()
C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1388450053.py:24: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: x['absolute_error_AF'].sum() / x['Actual'].sum()
C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1388450053.py:24: RuntimeWarning: divide by zero encountered in scalar divide
  lambda x: x['absolute_error_AF'].sum() / x['Actual'].sum()
C:\Users\Poovesh Kumar M\AppData\Local\Temp\ipykernel_13084\1388450053.py:29: RuntimeWarning: invalid value encountered in scalar divide
  lambda x: x['absolute_error_PF'].sum() / x['Actual'].sum()
C:\Users

In [120]:
df_data_wape
# wape_per_asin_AP
# wape_per_asin_AF
# wape_per_asin_PF

# Check the result
# print(wape_per_asin_PF[['asin', 'WAPE', 'wape_bucket']])

,brand,planning_group,asin,planned_qty,Actual,forecasted_sales,absolute_error_AP,absolute_error_AF,absolute_error_PF
0,Majestic Pure,AMAZON_US,B07G7QVSZT,1576,668,0.0,908,668.0,1576.0
1,Majestic Pure,AMAZON_US,B00UREAGU8,1242,511,0.0,731,511.0,1242.0
2,Majestic Pure,AMAZON_US,B0B9194J2B,11,0,0.0,11,0.0,11.0
3,Majestic Pure,AMAZON_US,B09GV1ZCSX,0,0,0.0,0,0.0,0.0
4,Majestic Pure,AMAZON_US,B09JL834PX,0,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
276,Majestic Pure,AMAZON_US,B09FNCBK5V,0,0,NaN,0,NaN,NaN
277,Majestic Pure,AMAZON_US,B08L3ZZFGB,0,0,NaN,0,NaN,NaN
278,Majestic Pure,AMAZON_US,B01BHO79V0,0,0,NaN,0,NaN,NaN
279,Majestic Pure,AMAZON_US,B07XKBVRXW,0,0,NaN,0,NaN,NaN


In [115]:
# Wape_buckets
df = wape_per_asin_PF


# Assuming df has columns: ['asin', 'WAPE']

conditions = [
    (df['WAPE'] <= 0.10),
    (df['WAPE'] > 0.10) & (df['WAPE'] <= 0.20),
    (df['WAPE'] > 0.20) & (df['WAPE'] <= 0.30),
    (df['WAPE'] > 0.30) & (df['WAPE'] <= 0.40),
    (df['WAPE'] > 0.40) & (df['WAPE'] <= 0.50),
    (df['WAPE'] > 0.50) & (df['WAPE'] <= 0.60),
    (df['WAPE'] > 0.60) & (df['WAPE'] <= 0.70),
    (df['WAPE'] > 0.70) & (df['WAPE'] <= 0.80),
    (df['WAPE'] > 0.80) & (df['WAPE'] <= 0.90),
    (df['WAPE'] > 0.90) & (df['WAPE'] <= 1.00),
    (df['WAPE'] > 1.00)
]

choices = [
    "A. 0 - 10%",
    "B. > 10% - 20%",
    "C. > 20% - 30%",
    "D. > 30% - 40%",
    "E. > 40% - 50%",
    "F. > 50% - 60%",
    "G. > 60% - 70%",
    "H. > 70% - 80%",
    "I. > 80% - 90%", 
    "J > 90% - 100%",
    "K.> 100%"
]

df['wape_bucket'] = np.select(conditions, choices, default='Other')


# Summarize the count of ASINs in each WAPE bucket
bucket_summary = df.groupby('wape_bucket').size().reset_index(name='asin_count')

# Sort the result for a cleaner view (optional)
bucket_summary = bucket_summary.sort_values(by='wape_bucket')

# Display the summary
print(bucket_summary)


# Save the forecast results to a CSV file
df.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/WAPE_summary_PF.csv', index=False)
bucket_summary.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/bucket_summary_PF.csv', index=False)


       wape_bucket  asin_count
0       A. 0 - 10%          89
1   B. > 10% - 20%          22
2   C. > 20% - 30%          10
3   D. > 30% - 40%           8
4   E. > 40% - 50%           4
5   F. > 50% - 60%          11
6   G. > 60% - 70%           4
7   H. > 70% - 80%           2
8   I. > 80% - 90%           1
9   J > 90% - 100%           2
10        K.> 100%          22
11           Other         106


In [22]:
%%time

# Forecasting organic_gmv_units


import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, r2_score

# Define exogenous variables
exogenous_vars = ['asp_week', 'ad_spend']  # Adjust this to match your DataFrame column names

# Prepare dictionaries to store results
X_train_fb2 = {}
X_test_fb2 = {}
y_test_fb_pred2 = {}

for asin in train_df['asin'].unique():
    print(f"Processing ASIN: {asin}")
    
    # Filter the training data for the current ASIN
    asin_train_data = train_df[train_df['asin'] == asin]
    asin_test_data = test_df[test_df['asin'] == asin]
    
    if asin_train_data.empty or asin_test_data.empty:
        print(f"No data found for ASIN: {asin}")
        continue
    
    # Prepare the training data for Prophet
    X_train_fb2[asin] = pd.DataFrame()
    X_train_fb2[asin]['ds'] = asin_train_data['Week_Start_date']
    X_train_fb2[asin]['y'] = asin_train_data['organic_gmv_units']
    
    # Add exogenous variables
    X_train_fb2[asin][exogenous_vars] = asin_train_data[exogenous_vars]
    
    # Ensure data types are correct
    X_train_fb2[asin]['ds'] = pd.to_datetime(X_train_fb2[asin]['ds'])
    X_train_fb2[asin]['y'] = X_train_fb2[asin]['y'].astype(float)
    
    # Initialize the Prophet model
    model = Prophet(changepoint_prior_scale=0.03)
    
    # Add exogenous variables as regressors with standardization
    for var in exogenous_vars:
        model.add_regressor(var, standardize=True)
    
    # Fit the model
    try:
        model.fit(X_train_fb2[asin])
    except ValueError as e:
        print(f"Error fitting model for ASIN {asin}: {e}")
        continue
    
    # Prepare the test data for Prophet
    X_test_fb2[asin] = pd.DataFrame()
    X_test_fb2[asin]['ds'] = asin_test_data['Week_Start_date']
    X_test_fb2[asin][exogenous_vars] = asin_test_data[exogenous_vars]
    
    # Ensure test data type is correct
    X_test_fb2[asin]['ds'] = pd.to_datetime(X_test_fb2[asin]['ds'])
    
    # Make predictions
    try:
        forecast_test = model.predict(X_test_fb2[asin])
    except ValueError as e:
        print(f"Error generating forecast for ASIN {asin}: {e}")
        continue
    
    # Store the predictions
    y_test_fb_pred2[asin] = forecast_test['yhat'].values
    y_test_fb_pred2[asin][y_test_fb_pred2[asin]<0] = 0
    y_test_fb_pred2[asin] = y_test_fb_pred2[asin].round(0).astype('int64')
    
# Create DataFrame with actual and forecasted values
df_results_org = []
for asin in y_test_fb_pred2:
    asin_test_data = test_df[test_df['asin'] == asin]
    df_results_org.append(pd.DataFrame({
        'asin': asin,
        'Week_Start_date': asin_test_data['Week_Start_date'],
        'org_actual_outlier_treated': asin_test_data['organic_gmv_units'],
        'forecasted_value': y_test_fb_pred2[asin]
    }))

df_results_org = pd.concat(df_results_org, ignore_index=True)



Processing ASIN: B00PMR3QF2


16:23:57 - cmdstanpy - INFO - Chain [1] start processing
16:23:57 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B00PV15BPW


16:23:58 - cmdstanpy - INFO - Chain [1] start processing
16:23:58 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B00Q7Z1UZ6
No data found for ASIN: B00Q7Z1UZ6
Processing ASIN: B00Q96XGUU


...

Processing ASIN: B0CQH9FHZT


16:27:26 - cmdstanpy - INFO - Chain [1] start processing
16:27:26 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B0CQHC2L5J


16:27:27 - cmdstanpy - INFO - Chain [1] start processing
16:27:27 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B0CXDH2LPP
Processing ASIN: B0CXDHX2QN
Processing ASIN: B0CXDRY3WC
Processing ASIN: B0CXDSCVJB
Processing ASIN: B0D1XNW89X
Error fitting model for ASIN B0D1XNW89X: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D4VHWB3C
Error fitting model for ASIN B0D4VHWB3C: Dataframe has less than 2 non-NaN rows.
CPU times: total: 42.4 s
Wall time: 3min 32s


In [23]:
# Storing Results in a DataFrame and Calculating Metrics:

df_base2 = df_base[['asin', 'Week_Start_date','organic_gmv_units']]

df_results_org1 =  pd.merge(df_results_org, df_base2, left_on=['asin', 'Week_Start_date'],
                     right_on=['asin', 'Week_Start_date'])


df_results_org2 = df_asin_vb[['asin', 'velocity_bucket']].merge(df_results_org1,left_on='asin', right_on='asin', how='right')

# Calculate Absolute Percentage Error
df_results_org2['absolute_percentage_error'] = abs((df_results_org2['forecasted_value'] - df_results_org2['organic_gmv_units']) / df_results_org2['organic_gmv_units'])

# Calculate Absolute Errors
df_results_org2['absolute_error'] = abs(df_results_org2['forecasted_value'] - df_results_org2['organic_gmv_units'])



# Calculate overall WAPE
overall_wape = df_results_org2['absolute_error'].sum() / df_results_org2['organic_gmv_units'].sum()
print(f"\nOverall WAPE: {overall_wape:.2%}")


# Calculate WAPE for each date and velocity bucket
wape_per_date_org = df_results_org2.groupby(['Week_Start_date']).apply(
    lambda x: x['absolute_error'].sum() / x['organic_gmv_units'].sum()
).reset_index(name='WAPE')
# Save the forecast results to a CSV file
wape_per_date_org.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_ORG_units.csv', index=False)

# Display WAPE
print("\nWAPE for each forecast date:")
print(wape_per_date_org)


# Calculate WAPE for each date and velocity bucket
wape_per_date_bucket_org = df_results_org2.groupby(['Week_Start_date', 'velocity_bucket']).apply(
    lambda x: x['absolute_error'].sum() / x['organic_gmv_units'].sum()
).reset_index(name='WAPE')



# Display WAPE
print("\nWAPE for each forecast date:")
print(wape_per_date_bucket_org)

# Save the forecast results to a CSV file
wape_per_date_bucket_org.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_buckets_ORG_units.csv', index=False)



Overall WAPE: 47.66%

WAPE for each forecast date:
   Week_Start_date      WAPE
0       2024-05-06  0.345270
1       2024-05-13  0.352903
2       2024-05-20  0.589735
3       2024-05-27  0.413040
4       2024-06-03  0.366230
5       2024-06-10  0.409163
6       2024-06-17  0.463482
7       2024-06-24  0.508216
8       2024-07-01  0.498013
9       2024-07-08  0.505587
10      2024-07-15  0.615651
11      2024-07-22  0.444107
12      2024-07-29  0.558417
13      2024-08-05  0.478128
14      2024-08-12  0.405726
15      2024-08-19  0.415344

WAPE for each forecast date:
   Week_Start_date velocity_bucket      WAPE
0       2024-05-06            1_SF  0.207440
1       2024-05-06             2_F  0.422297
2       2024-05-06             3_M  0.386688
3       2024-05-06            4_SM  0.401021
4       2024-05-13            1_SF  0.254404
..             ...             ...       ...
59      2024-08-12            4_SM  0.589731
60      2024-08-19            1_SF  0.194910
61      2024-08-19  

In [24]:
df_results_org

,asin,Week_Start_date,org_actual_outlier_treated,forecasted_value
0,B00PMR3QF2,2024-05-06,2279.6918,1805
1,B00PMR3QF2,2024-05-13,988.9535,945
2,B00PMR3QF2,2024-05-20,816.4571,1056
3,B00PMR3QF2,2024-05-27,743.9493,967
4,B00PMR3QF2,2024-06-03,743.9493,1967
...,...,...,...,...
4388,B0CXDSCVJB,2024-07-22,0.0000,0
4389,B0CXDSCVJB,2024-07-29,0.0000,0
4390,B0CXDSCVJB,2024-08-05,0.0000,0
4391,B0CXDSCVJB,2024-08-12,0.0000,0


In [25]:
%%time
# Forecasting inorganic_gmv_units

import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, r2_score

# Define exogenous variables
exogenous_vars = ['asp_week', 'ad_spend']  # Adjust this to match your DataFrame column names

# Prepare dictionaries to store results
X_train_fb2 = {}
X_test_fb2 = {}
y_train_fb_pred2 = {}
y_test_fb_pred2 = {}


for asin in train_df['asin'].unique():
    print(f"Processing ASIN: {asin}")
    
    # Filter the training data for the current ASIN
    asin_train_data = train_df[train_df['asin'] == asin]
    asin_test_data = test_df[test_df['asin'] == asin]
    
    if asin_train_data.empty or asin_test_data.empty:
        print(f"No data found for ASIN: {asin}")
        continue
    
    # Prepare the training data for Prophet
    X_train_fb2[asin] = pd.DataFrame()
    X_train_fb2[asin]['ds'] = asin_train_data['Week_Start_date']
    X_train_fb2[asin]['y'] = asin_train_data['inorganic_gmv_units']
    
    # Add exogenous variables
    X_train_fb2[asin][exogenous_vars] = asin_train_data[exogenous_vars]
    
    # Ensure data types are correct
    X_train_fb2[asin]['ds'] = pd.to_datetime(X_train_fb2[asin]['ds'])
    X_train_fb2[asin]['y'] = X_train_fb2[asin]['y'].astype(float)
    
    # Initialize the Prophet model
    model = Prophet(changepoint_prior_scale=0.03)
    
    # Prophet(interval_width=0.95, yearly_seasonality='auto', weekly_seasonality='auto',changepoint_prior_scale=0.03)
    
    # Add exogenous variables as regressors with standardization
    for var in exogenous_vars:
        model.add_regressor(var, standardize=True)
    
    # Fit the model
    try:
        model.fit(X_train_fb2[asin])
    except ValueError as e:
        print(f"Error fitting model for ASIN {asin}: {e}")
        continue
    
    # Prepare the test data for Prophet
    X_test_fb2[asin] = pd.DataFrame()
    X_test_fb2[asin]['ds'] = asin_test_data['Week_Start_date']
    X_test_fb2[asin][exogenous_vars] = asin_test_data[exogenous_vars]
    
    # Ensure test data type is correct
    X_test_fb2[asin]['ds'] = pd.to_datetime(X_test_fb2[asin]['ds'])
    
    # Make predictions
    try:
        forecast_train = model.predict(X_train_fb2[asin].drop(columns=['y']))
        forecast_test = model.predict(X_test_fb2[asin])
    except ValueError as e:
        print(f"Error generating forecast for ASIN {asin}: {e}")
        continue
    
    # Store the predictions
    y_test_fb_pred2[asin] = forecast_test['yhat'].values
    y_test_fb_pred2[asin][y_test_fb_pred2[asin]<0] = 0
    y_test_fb_pred2[asin] = y_test_fb_pred2[asin].round(0).astype('int64')
    
# Create DataFrame with actual and forecasted values
df_results_iorg = []
for asin in y_test_fb_pred2:
    asin_test_data = test_df[test_df['asin'] == asin]
    df_results_iorg.append(pd.DataFrame({
        'asin': asin,
        'Week_Start_date': asin_test_data['Week_Start_date'],
        'org_actual_outlier_treated': asin_test_data['inorganic_gmv_units'],
        'forecasted_value': y_test_fb_pred2[asin]
    }))

df_results_iorg = pd.concat(df_results_iorg, ignore_index=True)




Processing ASIN: B00PMR3QF2


16:27:30 - cmdstanpy - INFO - Chain [1] start processing
16:27:30 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B00PV15BPW


16:27:31 - cmdstanpy - INFO - Chain [1] start processing
16:27:31 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B00Q7Z1UZ6
No data found for ASIN: B00Q7Z1UZ6
Processing ASIN: B00Q96XGUU


...

Processing ASIN: B0CQH9FHZT


16:30:55 - cmdstanpy - INFO - Chain [1] start processing
16:30:56 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B0CQHC2L5J


16:30:56 - cmdstanpy - INFO - Chain [1] start processing
16:30:56 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B0CXDH2LPP
Processing ASIN: B0CXDHX2QN
Processing ASIN: B0CXDRY3WC
Processing ASIN: B0CXDSCVJB
Processing ASIN: B0D1XNW89X
Error fitting model for ASIN B0D1XNW89X: Dataframe has less than 2 non-NaN rows.
Processing ASIN: B0D4VHWB3C
Error fitting model for ASIN B0D4VHWB3C: Dataframe has less than 2 non-NaN rows.
CPU times: total: 58.7 s
Wall time: 3min 29s


In [26]:
# Storing Results in a DataFrame and Calculating Metrics:
df_base3 = df_base[['asin', 'Week_Start_date','inorganic_gmv_units']]

df_results_iorg1 =  pd.merge(df_results_iorg, df_base3, left_on=['asin', 'Week_Start_date'],
                     right_on=['asin', 'Week_Start_date'])


df_results_iorg2 = df_asin_vb[['asin', 'velocity_bucket']].merge(df_results_iorg1,left_on='asin', right_on='asin', how='right')

# Calculate Absolute Percentage Error
df_results_iorg2['absolute_percentage_error'] = abs((df_results_iorg2['forecasted_value'] - df_results_iorg2['inorganic_gmv_units']) / df_results_iorg2['inorganic_gmv_units'])

# Calculate Absolute Errors
df_results_iorg2['absolute_error'] = abs(df_results_iorg2['forecasted_value'] - df_results_iorg2['inorganic_gmv_units'])


# Calculate overall WAPE
overall_wape = df_results_iorg2['absolute_error'].sum() / df_results_iorg2['inorganic_gmv_units'].sum()
print(f"\nOverall WAPE: {overall_wape:.2%}")

# df_results_iorg2
# Calculate WAPE for each date and velocity bucket
wape_per_date_iorg = df_results_iorg2.groupby(['Week_Start_date']).apply(
    lambda x: x['absolute_error'].sum() / x['inorganic_gmv_units'].sum()
).reset_index(name='WAPE')
# Save the forecast results to a CSV file
wape_per_date_iorg.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_IORG_units.csv', index=False)

# Display WAPE
print("\nWAPE for each forecast date:")
print(wape_per_date_iorg)

# Calculate WAPE for each date and velocity bucket
wape_per_date_bucket_iorg = df_results_iorg2.groupby(['Week_Start_date', 'velocity_bucket']).apply(
    lambda x: x['absolute_error'].sum() / x['inorganic_gmv_units'].sum()
).reset_index(name='WAPE')

# Display WAPE
print("\nWAPE for each forecast date:")
print(wape_per_date_bucket_iorg)

# Save the forecast results to a CSV file
wape_per_date_bucket_iorg.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Weekly_WAPE_buckets_IORG_units.csv', index=False)



Overall WAPE: 29.39%

WAPE for each forecast date:
   Week_Start_date      WAPE
0       2024-05-06  0.206739
1       2024-05-13  0.209757
2       2024-05-20  0.396451
3       2024-05-27  0.306565
4       2024-06-03  0.262960
5       2024-06-10  0.284743
6       2024-06-17  0.323269
7       2024-06-24  0.234947
8       2024-07-01  0.314039
9       2024-07-08  0.226549
10      2024-07-15  0.398773
11      2024-07-22  0.307920
12      2024-07-29  0.379285
13      2024-08-05  0.221436
14      2024-08-12  0.275928
15      2024-08-19  0.266765

WAPE for each forecast date:
   Week_Start_date velocity_bucket      WAPE
0       2024-05-06            1_SF  0.181608
1       2024-05-06             2_F  0.184207
2       2024-05-06             3_M  0.186708
3       2024-05-06            4_SM  0.288613
4       2024-05-13            1_SF  0.128563
..             ...             ...       ...
59      2024-08-12            4_SM  0.643694
60      2024-08-19            1_SF  0.147455
61      2024-08-19  

In [27]:
# df_results_org2
df_results2
# df_results_iorg2


,asin,velocity_bucket,Week_Start_date,actual_outlier_treated,forecasted_value,gmv_units,absolute_percentage_error,absolute_error
0,B00PMR3QF2,1_SF,2024-05-06,1951.0,3614,5272,0.314492,1658
1,B00PMR3QF2,1_SF,2024-05-13,1774.0,1639,1774,0.076099,135
2,B00PMR3QF2,1_SF,2024-05-20,1100.0,1486,1100,0.350909,386
3,B00PMR3QF2,1_SF,2024-05-27,1232.0,1617,1232,0.312500,385
4,B00PMR3QF2,1_SF,2024-06-03,1232.0,3446,4933,0.301439,1487
...,...,...,...,...,...,...,...,...
4388,B0CXDSCVJB,4_SM,2024-07-22,0.0,0,0,NaN,0
4389,B0CXDSCVJB,4_SM,2024-07-29,0.0,0,0,NaN,0
4390,B0CXDSCVJB,4_SM,2024-08-05,0.0,0,0,NaN,0
4391,B0CXDSCVJB,4_SM,2024-08-12,0.0,0,0,NaN,0


In [28]:
# Result Dataframe:


sf_base = df_base[['velocity_bucket','asin','Week_Start_date', 'gmv_units', 'organic_gmv_units', 'inorganic_gmv_units']]

sf_base = sf_base[(sf_base['Week_Start_date'] >= '2024-05-06') & (sf_base['Week_Start_date'] <= '2024-08-19')]

sf_base = sf_base.rename(columns={'gmv_units': 'sales','organic_gmv_units': 'organic_sales','inorganic_gmv_units': 'inorganic_sales'})

df_results3 = df_results2.rename(columns={
    'absolute_percentage_error': 'abs_per_error_sales',
    'absolute_error': 'absolute_error_sales',
    'actual_outlier_treated': 'outlier_treated_sales',
    'Week_Start_date': 'forecast_date',
    'forecasted_value':'forecasted_sales'
    
})

df_results3 = df_results3[['asin', 'forecast_date','outlier_treated_sales','forecasted_sales','abs_per_error_sales', 'absolute_error_sales']]

#organic sales
df_results_org3 = df_results_org2.rename(columns={
    'absolute_percentage_error': 'abs_per_error_organic_sales',
    'absolute_error': 'absolute_error_organic_sales',
    'org_actual_outlier_treated': 'outlier_treated_organic_sales',
    'Week_Start_date': 'forecast_date',
    'forecasted_value':'forecasted_organic_sales'
})

df_results_org3 = df_results_org3[['asin', 'forecast_date','outlier_treated_organic_sales','forecasted_organic_sales','abs_per_error_organic_sales', 'absolute_error_organic_sales']]

#inorganic sales
#organic sales
df_results_iorg3 = df_results_iorg2.rename(columns={
    'absolute_percentage_error': 'abs_per_error_inorganic_sales',
    'absolute_error': 'absolute_error_inorganic_sales',
    'org_actual_outlier_treated': 'outlier_treated_inorganic_sales',
    'Week_Start_date': 'forecast_date',
    'forecasted_value':'forecasted_inorganic_sales'
})

df_results_iorg3 = df_results_iorg3[['asin', 'forecast_date','outlier_treated_inorganic_sales','forecasted_inorganic_sales','abs_per_error_inorganic_sales', 'absolute_error_inorganic_sales']]




#merging all df

sf_data = pd.merge(sf_base, df_results3, left_on=['asin', 'Week_Start_date'], right_on=['asin', 'forecast_date'])
sf_data = pd.merge(sf_data, df_results_org3, left_on=['asin', 'Week_Start_date'], right_on=['asin', 'forecast_date'])
sf_data = pd.merge(sf_data, df_results_iorg3, left_on=['asin', 'Week_Start_date'], right_on=['asin', 'forecast_date'])




# Display the first few rows of the results
sf_data = sf_data [['velocity_bucket','asin','Week_Start_date',
                    'sales','outlier_treated_sales','forecasted_sales','abs_per_error_sales', 'absolute_error_sales',
                    'organic_sales','outlier_treated_organic_sales','forecasted_organic_sales','abs_per_error_organic_sales', 'absolute_error_organic_sales',
                    'inorganic_sales','outlier_treated_inorganic_sales','forecasted_inorganic_sales','abs_per_error_inorganic_sales', 'absolute_error_inorganic_sales']]


In [29]:
print(sf_data.columns)


Index(['velocity_bucket', 'asin', 'Week_Start_date', 'sales',
       'outlier_treated_sales', 'forecasted_sales', 'abs_per_error_sales',
       'absolute_error_sales', 'organic_sales',
       'outlier_treated_organic_sales', 'forecasted_organic_sales',
       'abs_per_error_organic_sales', 'absolute_error_organic_sales',
       'inorganic_sales', 'outlier_treated_inorganic_sales',
       'forecasted_inorganic_sales', 'abs_per_error_inorganic_sales',
       'absolute_error_inorganic_sales'],
      dtype='object')


In [30]:
# Step 3: Display the Results & write into csv

sf_data.to_csv('C:/Users/Poovesh Kumar M/Downloads/Planning/Forecast_data.csv', index=False)


In [38]:
# before and after standardization for the selected ASINs
import pandas as pd
from prophet import Prophet

# Define exogenous variables and the ASINs of interest
exogenous_vars = ['asp_week', 'ad_spend']
selected_asins = ['B07JD9FS28', 'B079ZN8SC9', 'B09S26ZF1N', 'B09PZSKJ5N']

# List to store results
results = []

for asin in selected_asins:
    print(f"Processing ASIN: {asin}")
    
    # Filter the data for the current ASIN
    asin_train_data = train_df[train_df['asin'] == asin]
    
    if asin_train_data.empty:
        print(f"No data found for ASIN: {asin}")
        continue
    
    # Prepare the training data for Prophet
    df_train = pd.DataFrame()
    df_train['ds'] = asin_train_data['Week_Start_date']
    df_train['y'] = asin_train_data['gmv_units']
    
    # Add exogenous variables
    for var in exogenous_vars:
        df_train[var] = asin_train_data[var]
    
    # Initialize the Prophet model
    model = Prophet(changepoint_prior_scale=0.03)
    
    # Add exogenous variables as regressors with standardization
    for var in exogenous_vars:
        model.add_regressor(var, standardize=True)
    
    # Fit the model
    model.fit(df_train)
    
    # Access the model's scaling parameters
    scaling_params = model.extra_regressors
    
    # Compare original and standardized values
    for i in range(len(df_train)):
        result = {'asin': asin, 'Week_Start_date': df_train.iloc[i]['ds']}
        for var in exogenous_vars:
            original_value = df_train.iloc[i][var]
            mean = scaling_params[var]['mu']
            std = scaling_params[var]['std']
            standardized_value = (original_value - mean) / std
            
            result[f'before_{var}'] = original_value
            result[f'standardized_{var}'] = standardized_value
        
        results.append(result)

# Create DataFrame from the results list
df_comparison = pd.DataFrame(results)

# Display the resulting DataFrame
print(df_comparison.head())


Processing ASIN: B07JD9FS28


18:12:44 - cmdstanpy - INFO - Chain [1] start processing
18:12:44 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B079ZN8SC9


18:12:45 - cmdstanpy - INFO - Chain [1] start processing
18:12:45 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B09S26ZF1N


18:12:45 - cmdstanpy - INFO - Chain [1] start processing
18:12:46 - cmdstanpy - INFO - Chain [1] done processing


Processing ASIN: B09PZSKJ5N


18:12:46 - cmdstanpy - INFO - Chain [1] start processing
18:12:46 - cmdstanpy - INFO - Chain [1] done processing


         asin Week_Start_date  before_asp_week  standardized_asp_week  \
0  B07JD9FS28      2022-09-26        21.879300               4.148622   
1  B07JD9FS28      2022-10-03        16.085311              -0.003976   
2  B07JD9FS28      2022-10-10        16.017693              -0.052439   
3  B07JD9FS28      2022-10-17        16.033076              -0.041414   
4  B07JD9FS28      2022-10-24        15.994872              -0.068795   

   before_ad_spend  standardized_ad_spend  
0        1304.7274              -2.053665  
1        3717.3216              -0.484207  
2        4856.6743               0.256973  
3        4400.9435              -0.039492  
4        4056.4470              -0.263597  
